# Our Fuzzy IMDB-rating system  

The different types of membership functions

In [30]:
import math
import numpy as np
from collections import defaultdict, Counter


#Needs Float input
class TriangularMF:
    def __init__(self, name, start, top, end):
        self.name = name
        self.start = float(start)
        self.top = float(top)
        self.end = float(end)

    def calculate_membership(self, x):
        if x < self.start or x > self.end:
            return(0.0)
        if x < self.top:
            return (x-self.start)/(self.top-self.start)
        if x > self.top:
            return (self.end-x)/(self.end-self.top)
        if x == self.top:
            return 1.0
    
    def get_range(self):
        return(self.start, self.end)
        
#Needs Float input
class TrapezoidalMF:
    def __init__(self, name, start, left_top, right_top, end):
        self.name = name
        self.start = float(start)
        self.left_top = float(left_top)
        self.right_top = float(right_top)
        self.end = float(end)

    def calculate_membership(self, x):
        if x < self.start or x > self.end:
            return 0.0
        if x < self.left_top:
            return (x-self.start)/(self.left_top - self.start)
        if x > self.right_top:
            return (self.end-x)/(self.end-self.right_top)
        if x >= self.left_top and x <= self.right_top:
            return 1.0

    def get_range(self):
        return(self.start, self.end)
    
class GaussianMF:
    def __init__(self, name, mean, std):
        self.name = name
        self.mean = mean
        self.std = std

    def calculate_membership(self, x):
        exp = (-1*(x-self.mean)**2)/(2*self.std**2)
        return math.exp(exp)
        #     def calculate_membership(self, x):
#         first = float(1)/(math.sqrt(2*math.pi)*self.std)
#         second = (x-self.mean)**2
#         second = second/(2*(self.std**2))
#         return(first * math.exp(-second))

    # Calculate range as that of 99.7% of the entire fuzzy set (3 standard deviations)
    def get_range(self):
        start = self.mean - 3*self.std
        end = self.mean + 3*self.std
        return(start, end)


In [33]:
g = GaussianMF("gaus", 10, 2)
print(g.calculate_membership(8))
print(g.get_range())

0.6065306597126334
(4, 16)


The Variable-class

In [34]:
class Variable:
    """General class for variables in an FLS."""
    def __init__(self, name, range, mfs):
        self.name = name
        self.range = range
        self.mfs = mfs

    def calculate_memberships(self, x):
        """Test function to check whether
        you put together the right mfs in your variables."""
        return {
            mf.name : mf.calculate_membership(x)
            for mf in self.mfs
        }

    def get_mf_by_name(self, name):
        for mf in self.mfs:
            if mf.name == name:
                return mf

class Input(Variable):
    """Class for input variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "input"

class Output(Variable):
    """Class for output variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "output"

Hier moeten we variabelen en hun Membership-functies gaan definieren

In [19]:
# Number of Movies for actors
few = TrapezoidalMF('few', 0, 0, 3, 5)
medium = TriangularMF('medium', 3, 5, 7)
much = TriangularMF('much', 5, 7, 9)
very_much = TrapezoidalMF('very_much', 8, 10, 15, 15)
mfs_actor_movies = [few, medium, much, very_much]
actor_movies = Input('actor_movies', (0, 15), mfs_actor_movies)

actor_movies.calculate_memberships(4)

{'few': 0.5, 'medium': 0.5, 'much': 0.0, 'very_much': 0.0}

In [20]:
# Number of Movies for Director
few = TrapezoidalMF('few', 0, 0, 3, 5)
medium = TriangularMF('medium', 3, 5, 7)
much = TriangularMF('much', 5, 7, 9)
very_much = TrapezoidalMF('very_much', 8, 10, 15, 15)
mfs_director_movies = [few, medium, much, very_much]
director_movies = Input('director_movies', (0, 15), mfs_director_movies)

In [21]:
# Number of Movies for Composer
few = TrapezoidalMF('few', 0, 0, 3, 5)
medium = TriangularMF('medium', 3, 5, 7)
much = TriangularMF('much', 5, 7, 9)
very_much = TrapezoidalMF('very_much', 8, 10, 15, 15)
mfs_composer_movies = [few, medium, much, very_much]
composer_movies = Input('composer_movies', (0, 15), mfs_composer_movies)

In [22]:
# Number of Movies for Visual Effects superviser
few = TrapezoidalMF('few', 0, 0, 3, 5)
medium = TriangularMF('medium', 3, 5, 7)
much = TriangularMF('much', 5, 7, 9)
very_much = TrapezoidalMF('very_much', 8, 10, 15, 15)
mfs_ve_movies = [few, medium, much, very_much]
ve_movies = Input('ve_movies', (0, 15), mfs_ve_movies)

In [47]:
# Average movie rating
terrible = TrapezoidalMF('terrible', 0, 0, 3, 4)
bad = TriangularMF('bad', 3, 4.5, 6)
acceptable = TriangularMF('acceptable', 5.5, 6.5, 7.5)
good = TriangularMF('good', 7, 7.75, 8.5)
excellent = TrapezoidalMF('excellent', 7.5, 8, 10.0, 10.0)
mfs_average_movie_rating = [terrible, bad, acceptable, good, excellent]
average_movie_rating = Input('average_movie_rating', (0.0, 10.0), mfs_average_movie_rating)

average_movie_rating.calculate_memberships(5.9)

{'acceptable': 0.40000000000000036,
 'bad': 0.06666666666666643,
 'excellent': 0.0,
 'good': 0.0,
 'terrible': 0.0}

In [38]:
# Budget
low = GaussianMF('low', 0, 30)
medium = GaussianMF('medium', 80, 60)
high = GaussianMF('high', 430, 250)
mfs_budget = [low, medium, high]
budget = Input('average_movie_rating', (0, 430), mfs_budget)

print(budget.calculate_memberships(20))
print(budget.calculate_memberships(60))
print(budget.calculate_memberships(100))
print(budget.calculate_memberships(200))
print(budget.calculate_memberships(300))

{'high': 0.2605918210126892, 'low': 0.8007374029168081, 'medium': 0.6065306597126334}
{'high': 0.33447270571756405, 'low': 0.1353352832366127, 'medium': 0.9459594689067654}
{'high': 0.41844910891303544, 'low': 0.0038659201394728076, 'medium': 0.9459594689067654}
{'high': 0.6549476304858832, 'low': 2.233631436203166e-10, 'medium': 0.1353352832366127}
{'high': 0.8735411859788502, 'low': 1.9287498479639178e-22, 'medium': 0.001203859994828203}


In [73]:
# Actor
terrible = TrapezoidalMF('terrible', 0, 0, 3, 4)
bad = TriangularMF('bad', 3, 4.5, 6)
acceptable = TriangularMF('acceptable', 5.5, 6.5, 7.5)
good = TriangularMF('good', 7, 7.75, 8.5)
excellent = TrapezoidalMF('excellent', 7.5, 8, 10.0, 10.0)
mfs_actor= [terrible, bad, acceptable, good, excellent]
actor_output = Output('actor_output', (0.0, 10.0), mfs_actor)
actor_input = Input('actor_input', (0.0, 10.0), mfs_actor)


In [72]:
# Director
terrible = TrapezoidalMF('terrible', 0, 0, 3, 4)
bad = TriangularMF('bad', 3, 4.5, 6)
acceptable = TriangularMF('acceptable', 5.5, 6.5, 7.5)
good = TriangularMF('good', 7, 7.75, 8.5)
excellent = TrapezoidalMF('excellent', 7.5, 8, 10.0, 10.0)
mfs_director= [terrible, bad, acceptable, good, excellent]
director_output = Output('director_output', (0.0, 10.0), mfs_director)
director_input = Input('director_input', (0.0, 10.0), mfs_director)

In [71]:
# ve
terrible = TrapezoidalMF('terrible', 0, 0, 3, 4)
bad = TriangularMF('bad', 3, 4.5, 6)
acceptable = TriangularMF('acceptable', 5.5, 6.5, 7.5)
good = TriangularMF('good', 7, 7.75, 8.5)
excellent = TrapezoidalMF('excellent', 7.5, 8, 10.0, 10.0)
mfs_ve = [terrible, bad, acceptable, good, excellent]
ve_output = Output('ve_output', (0.0, 10.0), mfs_ve)
ve_input = Input('ve_input', (0.0, 10.0), mfs_ve)

In [70]:
# Performing
terrible = TrapezoidalMF('terrible', 0, 0, 3, 4)
bad = TriangularMF('bad', 3, 4.5, 6)
acceptable = TriangularMF('acceptable', 5.5, 6.5, 7.5)
good = TriangularMF('good', 7, 7.75, 8.5)
excellent = TrapezoidalMF('excellent', 7.5, 8, 10.0, 10.0)
mfs_performing= [terrible, bad, acceptable, good, excellent]
performing_output = Output('performing_output', (0.0, 10.0), mfs_performing)
performing_input = Input('performing_input', (0.0, 10.0), mfs_performing)#


In [69]:
#creating
terrible = TrapezoidalMF('terrible', 0, 0, 3, 4)
bad = TriangularMF('bad', 3, 4.5, 6)
acceptable = TriangularMF('acceptable', 5.5, 6.5, 7.5)
good = TriangularMF('good', 7, 7.75, 8.5)
excellent = TrapezoidalMF('excellent', 7.5, 8, 10.0, 10.0)
mfs_creating= [terrible, bad, acceptable, good, excellent]
creating_output = Output('creating_output', (0.0, 10.0), mfs_creating)
creating_input = Input('creating_input', (0.0, 10.0), mfs_creating)

The Rule-class

In [6]:
class Rule:
    """Fuzzy rule class, initialized with an antecedent (list of strings),
    operator (string) and consequent (string)."""
    def __init__(self, n, antecedent, operator, consequent):
        self.number = n
        self.antecedent = antecedent
        self.operator = operator
        self.consequent = consequent
        self.firing_strength = 0

    def calculate_firing_strength(self, datapoint, inputs):
        fs = []
        for index, variable in enumerate(inputs):
#            print(index)
            memb_dict = (variable.calculate_memberships(datapoint[index]))
            fs.append(memb_dict[self.antecedent[index]])
        if self.operator == 'and':
            self.firing_strength = min(fs)
        if self.operator == 'or':    
            self.firing_strength = max(fs)
        return self.firing_strength

Hier moeten we Rules gaan definieren

In [64]:
#System1: Inferencing actor-quality
rule1 = Rule(1, ["terrible", "few"], "and", "terrible")
rule2 = Rule(1, ["terrible", "medium"], "and", "terrible")
rule3 = Rule(1, ["terrible", "much"], "and", "terrible")
rule4 = Rule(1, ["terrible", "very_much"], "and", "bad")

rule5 = Rule(1, ["bad", "few"], "and", "terrible")
rule6 = Rule(1, ["bad", "medium"], "and", "bad")
rule7 = Rule(1, ["bad", "much"], "and", "bad")
rule8 = Rule(1, ["bad", "very_much"], "and", "acceptable")

rule9 = Rule(1, ["acceptable", "few"], "and", "bad")
rule10 = Rule(1, ["acceptable", "medium"], "and", "acceptable")
rule11 = Rule(1, ["acceptable", "much"], "and", "acceptable")
rule12 = Rule(1, ["acceptable", "very_much"], "and", "good")

rule13 = Rule(1, ["good", "few"], "and", "acceptable")
rule14 = Rule(1, ["good", "medium"], "and", "good")
rule15 = Rule(1, ["good", "much"], "and", "good")
rule16 = Rule(1, ["good", "very_much"], "and", "excellent")

rule17 = Rule(1, ["excellent", "few"], "and", "good")
rule18 = Rule(1, ["excellent", "medium"], "and", "excellent")
rule19 = Rule(1, ["excellent", "much"], "and", "excellent")
rule20 = Rule(1, ["excellent", "very_much"], "and", "excellent")

actor_rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10, rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20]

#Same as director-rulebase and ve-rulebase
actor_rulebase = Rulebase(actor_rules)
print(actor_rulebase.calculate_firing_strengths([7.5, 8], [average_movie_rating, actor_movies]))

Counter({'good': 0.5})


In [74]:
# Rulebase that takes actors and ve as input and (PERFORMING) as output
# Actor weighs more than VE at the moment
r1 = Rule(1, ["terrible", "terrible"], "and", "terrible")
r2 = Rule(1, ["terrible", "bad"], "and", "terrible")
r3 = Rule(1, ["terrible", "acceptable"], "and", "terrible")
r4 = Rule(1, ["terrible", "good"], "and", "bad")
r5 = Rule(1, ["terrible", "excellent"], "and", "bad")

r6 = Rule(1, ["bad", "terrible"], "and", "terrible")
r7 = Rule(1, ["bad", "bad"], "and", "bad")
r8 = Rule(1, ["bad", "acceptable"], "and", "bad")
r9 = Rule(1, ["bad", "good"], "and", "bad")
r10 = Rule(1, ["bad", "excellent"], "and", "acceptable")

r11 = Rule(1, ["acceptable", "terrible"], "and", "bad")
r12 = Rule(1, ["acceptable", "bad"], "and", "acceptable")
r13 = Rule(1, ["acceptable", "acceptable"], "and", "acceptable")
r14 = Rule(1, ["acceptable", "good"], "and", "acceptable")
r15 = Rule(1, ["acceptable", "excellent"], "and", "good")

r16 = Rule(1, ["good", "terrible"], "and", "acceptable")
r17 = Rule(1, ["good", "bad"], "and", "good")
r18 = Rule(1, ["good", "acceptable"], "and", "good")
r19 = Rule(1, ["good", "good"], "and", "good")
r20 = Rule(1, ["good", "excellent"], "and", "excellent")

r21 = Rule(1, ["excellent", "terrible"], "and", "good")
r22 = Rule(1, ["excellent", "bad"], "and", "good")
r23 = Rule(1, ["excellent", "acceptable"], "and", "excellent")
r24 = Rule(1, ["excellent", "good"], "and", "excellent")
r25 = Rule(1, ["excellent", "excellent"], "and", "excellent")

performing_rules = [r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12, r13, r14, r15, r16, r17, r18, r19, r20, r21, r22, r23, r24, r25]

#Can for now also be used as creating rulebase. But we might want to weigh the variables differently there.
performing_rulebase = Rulebase(performing_rules)
print(performing_rulebase.calculate_firing_strengths([7.5, 8], [actor_input, ve_input]))


Counter({'good': 0.6666666666666666, 'excellent': 0.6666666666666666})


The Rule-base Class

In [40]:
from collections import Counter

class Rulebase:
    """The fuzzy rulebase collects all rules for the FLS, can
    calculate the firing strengths of its rules."""
    def __init__(self, rules):
        self.rules = rules

    def calculate_firing_strengths(self, datapoint, inputs):
        result = Counter()
        for i, rule in enumerate(self.rules):
            fs = rule.calculate_firing_strength(datapoint, inputs)
            consequent = rule.consequent
            if fs > result[consequent]:
                result[consequent] = fs
        return result

The inference-algorithm. Support nu alleen nog Mandami-inference met 'smallest of max' en 'largest of max' als defuzzification.

In [7]:
class Reasoner:
    def __init__(self, rulebase, inputs, output, n_points, defuzzification):
        self.rulebase = rulebase
        self.inputs = inputs
        self.output = output
        self.discretize = n_points
        self.defuzzification = defuzzification

    def inference(self, datapoint):
        # 1. Calculate the highest firing strength found in the rules per 
        # membership function of the output variable
        # looks like: {"low":0.5, "medium":0.25, "high":0}        
        firing_strengths = (rulebase.calculate_firing_strengths(datapoint, self.inputs))
        print(firing_strengths)

        # 2. Aggragate and discretize
        # looks like: [(0.0, 1), (1.2437810945273631, 1), (2.4875621890547261, 1), (3.7313432835820892, 1), ...]
        input_value_pairs = self.aggregate(firing_strengths)
        
        # 3. Defuzzify
        # looks like a scalar
        crisp_output = self.defuzzify(input_value_pairs)
        return crisp_output

    def aggregate(self, firing_strengths):
        
        # First find where the aggrageted area starts and ends
        (start, end) = (-1, -1)
        for fs in firing_strengths:
            if firing_strengths[fs] > 0.0:
                mf = self.output.get_mf_by_name(fs)
                (s, e) = mf.get_range()
                if start == -1 or s <= start:
                    start = s
                if end == -1 or end <= e:
                    end = e
#        print(start, end)
        # Second discretize this area and aggragate      
        # Your code here
        step_size = (end-start)/(float(self.discretize) - 1.0)
        input_value_pairs = []
        while start <= end:
            memb = self.memb_of_agg(firing_strengths, start)
            input_value_pairs.append((start, memb))
            start += step_size
        return input_value_pairs
        

    def memb_of_agg(self, firing_strengths, x):
        strength = 0
        for fs in firing_strengths:
            mf = self.output.get_mf_by_name(fs)
            m = mf.calculate_membership(x)
            m = min(m, firing_strengths[fs])
            if m > strength:
                strength = m
        return strength

    def defuzzify(self, input_value_pairs):
        crisp = -1
        maximum = 0
        if self.defuzzification == 'som':
            for (i, v) in input_value_pairs:
                if v > maximum or crisp == -1:
                    crisp = i
                    maximum = v
        if self.defuzzification =='lom':
            for (i, v) in input_value_pairs:
                if v >= maximum or crisp == -1:
                    crisp = i
                    maximum = v
        return crisp